# Error propagation with Sympy

Sympy is a Python module for symbolic computation (like Mathematica and Matlab) with an elegant Python design.

It can easily do mechanical tasks such as computing first derivatives for error propagation. We use this here to write a generic error propagation function in only 20+ lines.

In [1]:
import sympy


def value_and_covariance_gen(expr, variables):
    expr = sympy.parse_expr(expr)

    symbols = sympy.symbols(variables)
    cov_symbols = sympy.symbols(tuple("C_" + k for k in variables))
    expr2 = sum(expr.diff(s) ** 2 * c for s, c in zip(symbols, cov_symbols))
    expr2 = expr2.simplify()

    fval = sympy.lambdify(symbols, expr)
    fcov = sympy.lambdify(symbols + cov_symbols, expr2)

    def fn(**kwargs):
        x = tuple(v[0] for v in kwargs.values())
        c = tuple(v[1] for v in kwargs.values())
        return fval(*x), fcov(*x, *c)

    return fn


def value_and_covariance(expr, **kwargs):
    return value_and_covariance_gen(expr, tuple(kwargs))(**kwargs)

That's all, folks!

`value_and_covariance_gen` generates a Python function that computes the value of the expression and the propagated covariance. `value_and_covariance` is just a shortcut to generate and immediately call the function. The generated Python code is fast to evaluate (it just calls numpy functions).

Generating the propagating function takes about 6ms on my computer. Evaluating it is very fast.

**Limitations** 
- Only uncorrelated inputs are currently supported
- Expression must consist of basic math

## Some examples

In [4]:
value_and_covariance("a + b", a=(1, 0.1), b=(2, 0.2))

(3, 0.30000000000000004)

In [3]:
value_and_covariance("s / (s + b)", s=(5, 0.5), b=(10, 0.1))

(0.3333333333333333, 0.001037037037037037)